In [1]:
import yaml
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader

In [2]:
with open('Config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)

In [3]:
gemini = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key='KEY',
    temperature=0.1
    )

In [4]:
def load_knowledge_base(file_path):
    """Carrega a base de conhecimento de um arquivo de texto"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        # Retorna conteúdo de exemplo se arquivo não existir
        return """Problemas Comuns em Controle de videogame:
Drift (movimento do analógico): Ocorre devido ao desgaste dos potenciômetros internos, que detectam o movimento.

Problemas de pareamento ou conexão: O controle pode não se conectar corretamente ao console ou PC.

Bateria fraca ou não carregando: As baterias podem estar gastas ou os contatos podem estar sujos ou danificados.

Botões travando ou não respondendo: Sujeira, cola ou desgaste podem causar mau funcionamento dos botões."""

In [5]:
#CARREGAR A BASE DE CONHECIMENTO
knowledge_base_content = load_knowledge_base('manual.txt')

#histórico de conversas simulado
historico_de_conversas = """Cliente: Meu controle está com defeito no analógico,preciso saber o que fazer para parar de puxar.
chatgui: Você já verificou se não tem poeira?
cliente: Sim, até limpei o analógico, mas ainda nao parou."""

#Nova pergunta do cliente
pergunta = "Meu controle está com defeito no analógico,preciso saber o que fazer para parar de puxar."

#preparar inputs
inputs = {
    "context": knowledge_base_content,
    "question":pergunta,
    "historico": historico_de_conversas
}

In [6]:
#Template para consulta na base de conhecimento
prompt_base_conhecimento = PromptTemplate(
    input_variables=["context","question"],
    template="""
    Para responder as perguntas baseadas no meu controle,
    responda com apenas informações fornecidas no manual enviado,
    seja específico e técnico mantendo a linguagem acessível.
    
    Base de conhecimento: {context}
    Pergunta ao cliente: {question}
    Resposta técnica baseada na base de conheimento:"""
)

# Template para análise do histórico
prompt_historico_de_conversas = PromptTemplate(
    input_variables=["historico", "question"],
    template="""Analise o histórico da conversa para entender o contexto do problema.
    Use essas informações para melhorar a sua resposta, e fornecer diversas opções
    para solucionar o meu problema.
   
    Histórico da Conversa: {historico}
   
    Pergunta Atual: {question}
   
    Análise contextual do histórico:"""
)


# Template para resposta final combinada
prompt_final = PromptTemplate(
    input_variables=["resposta_base_conhecimento", "resposta_historico_conversas"],
    template="""Junte as informações do manual fornecido e o contexto histórico para criar uma resposta completa,
    e me mostre todos os cuidados recomendáveis para se cuidar do controle, como dito no manual.
   
    Informações Técnicas: {resposta_base_conhecimento}
   
    Contexto da Conversa: {resposta_historico_de_conversas}
   
    Resposta Final Integrada:"""
)

In [7]:
print("=== CONFIGURAÇÃO CONCLUÍDA ===")
print(f"Base de conhecimento carregada: {len(knowledge_base_content)} caracteres")
print(f"Pergunta do cliente: {pergunta}")

#Criar as Chains
chain_base_conhecimento = prompt_base_conhecimento | gemini
chain_historico_de_conversas = prompt_historico_de_conversas | gemini
chain_final = prompt_final | gemini

#Executar primeira chaib -Base de conhecimento
print("1. Consultando base de conhecimento...")
resultado_base_conhecimento = chain_base_conhecimento.invoke({
    "context": inputs["context"],
    "question" : inputs["question"]
})

=== CONFIGURAÇÃO CONCLUÍDA ===
Base de conhecimento carregada: 475 caracteres
Pergunta do cliente: Meu controle está com defeito no analógico,preciso saber o que fazer para parar de puxar.
1. Consultando base de conhecimento...


In [8]:
# Executar segunda chain - Histórico
print("2. Analisando Histórico de converas . . .")
resultado_historico_de_conversas = chain_historico_de_conversas.invoke({
    "historico": inputs["historico"],
    "question" : inputs["question"]
})


2. Analisando Histórico de converas . . .


In [9]:
#Executar chain final - Combinação
print("3. Gerando resposta final integrada...")
resultado_final = chain_final.invoke({
    "resposta_base_conhecimento": resultado_base_conhecimento.content,
    "resposta_historico_de_conversas": resultado_historico_de_conversas.content
})

3. Gerando resposta final integrada...


In [10]:
print("\n ANÁLISE DA BASE DE CONHECIMENTO:")
print("-" * 40)
print(resultado_base_conhecimento.content)


 ANÁLISE DA BASE DE CONHECIMENTO:
----------------------------------------
O problema de "puxar" no analógico (drift) ocorre devido ao desgaste dos potenciômetros internos do controle. Esses componentes são responsáveis por detectar o movimento do analógico. Infelizmente, a base de conhecimento não oferece soluções para corrigir o problema de drift.


In [11]:
print("\n ANÁLISE DE HISTOTICO DE CONVERSAS:")
print("-" * 40)
print(resultado_historico_de_conversas.content)


 ANÁLISE DE HISTOTICO DE CONVERSAS:
----------------------------------------
O cliente está com um problema no analógico do controle, que está "puxando" (movimentando o cursor/personagem sozinho). Ele já tentou limpar o analógico, o que sugere que ele já tem alguma familiaridade com a solução mais básica. A pergunta atual é uma repetição da pergunta inicial, indicando que o problema persiste e ele busca soluções mais avançadas.

**Opções para solucionar o problema do analógico "puxando":**

Considerando que a limpeza não resolveu, aqui estão diversas opções, desde as mais simples até as mais complexas, e algumas que dependem do tipo de controle:

**Soluções Simples (e que você já pode ter tentado, mas vale a pena verificar):**

1.  **Calibração do Controle (no sistema):**
    *   **Windows:** Vá em "Dispositivos e Impressoras", clique com o botão direito no controle, selecione "Configurações de Controlador de Jogo", selecione o controle e clique em "Propriedades". Deve haver uma aba d

In [12]:
print("\n RESPOSTA FINAL PARA O CLIENTE:")
print("-" * 40)
print(resultado_final.content)


 RESPOSTA FINAL PARA O CLIENTE:
----------------------------------------
Com base nas informações técnicas fornecidas e no contexto da conversa, o problema de "drift" no analógico do controle é causado pelo desgaste dos potenciômetros internos. Infelizmente, o manual não oferece soluções diretas para corrigir esse problema. No entanto, podemos combinar essa informação com o contexto e oferecer uma resposta completa e cuidadosa ao cliente, focando em prevenção e alternativas:

**Resposta ao Cliente:**

"Entendo que você já tentou limpar o analógico e o problema persiste. De acordo com as informações técnicas que temos, o 'drift' é geralmente causado pelo desgaste natural dos componentes internos do analógico, chamados potenciômetros. Infelizmente, não há uma solução simples e garantida para esse tipo de problema.

**Cuidados Recomendáveis e Alternativas:**

Embora não possamos reverter o desgaste, podemos tentar algumas coisas e, no futuro, adotar práticas para prolongar a vida útil do